Osnabrück University - Machine Learning (Summer Term 2016) - Prof. Dr.-Ing. G. Heidemann, Ulf Krumnack

# Exercise Sheet 05

## Introduction

This week's sheet is supposed to last two weeks and thus should be solved and handed in before the end of **Sunday, May 22, 2016**. If you need help (and Google and other resources were not enough), feel free to contact your groups designated tutor or whomever of us you run into first. Please upload your results to your group's studip folder.

### SciPy and scikit-learn and playsound

From now on you will sometimes need the python package [scipy](https://pypi.python.org/pypi/scipy). To check if you already have a running version installed, run the following cell. If the output is `scipy not found` follow the instructions below to install it. Otherwise just skip the following paragraphs and continue with the assignments.

In [ ]:
import importlib
assert importlib.util.find_spec('scipy') is not None, 'scipy not found'

On Unix systems you can easily install it with `pip3 install scipy` from any terminal window. If it fails, try to figure out how to install a Fortran compiler for your OS or ask one of your fellow tutors for help.

On Windows it is a little bit more difficult to get a Fortran compiler (although [MinGW](http://www.mingw.org/) offers one it is still very difficult to get everything to run), so we recommend you to take the [precompiled binaries](http://www.lfd.uci.edu/~gohlke/pythonlibs/#scipy) of Christoph Gohlke. If you previously installed a 32bit version of Python download `scipy-0.17.0-cp35-none-win32.whl`, if you have a 64bit version please resort to `scipy-0.17.0-cp35-none-win_amd64.whl`. If you are unsure which version you run, run the following cell to figure it out:

In [ ]:
import platform
print('You are running a {} ({}) version.'.format(*platform.architecture()))

To install the binaries open your command line, navigate to your folder where you downloaded the `*.whl` file to (`cd FOLDER`) and run `pip install scipy-0.17.0-cp35-none-win32.whl` (or `pip install scipy-0.17.0-cp35-none-win_amd64.whl` if you downloaded the 64 bit version). If you run into troubles, get in touch with us!

## Assignment 1: Curse of Dimensionality [X Points]

## Assignment 2: Implement and apply PCA [X Points]

## Assignment 3: Whitening and Reconstruction of Data [X Points]

## Assignment 4: Eigenfaces [X Points]

## Assignment 5: Theory [X Points]